In [12]:
import pandas as pd
import numpy as np

In [13]:
articles_data_path = r'C:\Users\ayush\Documents\Columbia MS DS\2nd_SEM\Applied_ML\AML_Final_Project\Data\articles.csv'
customer_data_path = r'C:\Users\ayush\Documents\Columbia MS DS\2nd_SEM\Applied_ML\AML_Final_Project\Data\customers.csv'
transaction_data_path = r'C:\Users\ayush\Documents\Columbia MS DS\2nd_SEM\Applied_ML\AML_Final_Project\Data\transactions_train.csv'

In [14]:
customers_data = pd.read_csv(customer_data_path)
transaction_data = pd.read_csv(transaction_data_path)
articles_data = pd.read_csv(articles_data_path)



In [273]:
articles_data.shape

(105542, 11)

# reading preprocessed Data

In [53]:
cust_features = r"C:\Users\ayush\Documents\Columbia MS DS\2nd_SEM\Applied_ML\AML_Final_Project\h_m_generated_features\customer_features.csv"

customer_features = pd.read_csv(cust_features)
recent_transactio_article_id = pd.read_csv(r"C:\Users\ayush\Documents\Columbia MS DS\2nd_SEM\Applied_ML\AML_Final_Project\h_m_generated_features\recent_transactio_article_id.csv")
frequently_bought_item_id_count = pd.read_csv(r"C:\Users\ayush\Documents\Columbia MS DS\2nd_SEM\Applied_ML\AML_Final_Project\h_m_generated_features\frequently_bought_item_id_count.csv")
time_features = pd.read_csv(r"C:\Users\ayush\Documents\Columbia MS DS\2nd_SEM\Applied_ML\AML_Final_Project\h_m_generated_features\time_features.csv")
article_embedding_df = pd.read_csv(r"C:\Users\ayush\Documents\Columbia MS DS\2nd_SEM\Applied_ML\AML_Final_Project\h_m_generated_features\article_embedding.csv")

In [154]:
print(customer_features.columns)
print(recent_transactio_article_id.columns)
print(frequently_bought_item_id_count.columns)
print(time_features.columns)
print(article_embedding_df.columns)

Index(['Unnamed: 0', 'customer_id', 'visits', 'no_unique_items_bought',
       'total_items_bought', 'list_of_items_bought', 'total_monetary_value',
       'max_spent_on_item', 'recency', 'latency', 'avg_bought_per_visit'],
      dtype='object')
Index(['Unnamed: 0', 'customer_id', 'recent_transaction_article_id'], dtype='object')
Index(['Unnamed: 0', 'customer_id', 'article_id', 'count', 't_dat'], dtype='object')
Index(['customer_id', 'weekday', 'Season'], dtype='object')
Index(['Unnamed: 0', 'article_id', 'glove_vec0', 'glove_vec1', 'glove_vec2',
       'glove_vec3', 'glove_vec4', 'glove_vec5', 'glove_vec6', 'glove_vec7',
       ...
       'glove_vec90', 'glove_vec91', 'glove_vec92', 'glove_vec93',
       'glove_vec94', 'glove_vec95', 'glove_vec96', 'glove_vec97',
       'glove_vec98', 'glove_vec99'],
      dtype='object', length=102)


# Merging features

In [159]:
temp_1 = pd.merge(customer_features.drop('Unnamed: 0',axis = 1 ),recent_transactio_article_id.drop('Unnamed: 0',axis = 1), on = 'customer_id')
temp_2 = pd.merge(frequently_bought_item_id_count.drop('Unnamed: 0',axis = 1 ),temp_1, on = ['customer_id'])
temp_3 = pd.merge(temp_2,time_features, on = ['customer_id'])
features = pd.merge(temp_3, article_embedding_df, on = ['article_id'])

In [160]:
import gc

# freeing up memory
del temp_1
del temp_2
del temp_3
gc.collect()

182

In [161]:
features.drop('Unnamed: 0',axis = 1, inplace = True)

In [163]:
features['mean_glove'] = features.iloc[:,-1:-101:-1].mean(axis = 1)

In [164]:
features.drop(list(features.columns[-2:-102:-1]),axis = 1,inplace = True)

In [170]:
features

,customer_id,article_id,count,t_dat,visits,no_unique_items_bought,total_items_bought,list_of_items_bought,total_monetary_value,max_spent_on_item,recency,latency,avg_bought_per_visit,recent_transaction_article_id,weekday,Season,mean_glove
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,797065001,2,2019-09-28,10,19,21,"[625548001, 176209023, 627759010, 697138006, 5...",0.648983,0.054220,17,68.666667,2.100000,568601043,Saturday,Spring,0.010994
1,02c53384a09864305de849cb7f2238ce8d652e624142cb...,797065001,2,2019-10-07,4,13,14,"[799365003, 757903001, 758226001, 631744004, 7...",0.559729,0.067780,351,10.333333,3.500000,797065001,Friday,Fall,0.010994
2,083cc398f74528df2c2f3d45a56f2f73b57a47cd4fac30...,797065001,1,2019-09-09,1,1,1,[797065001],0.067780,0.067780,379,0.000000,1.000000,797065001,Monday,Fall,0.010994
3,08a8ed4aef457ee131546695aa1d7e01c9237bfeb1dfab...,797065001,1,2019-10-02,3,4,4,"[694124003, 721388005, 626442021, 797065001]",0.126797,0.067780,356,111.500000,1.333333,797065001,Thursday,Winter,0.010994
4,103709a80acb9e3dc4d95298dd31c39b5ce72ba26fcacf...,797065001,1,2019-11-02,1,1,1,[797065001],0.061000,0.061000,325,0.000000,1.000000,797065001,Saturday,Fall,0.010994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1362276,ffe701895a1475c53d88e2a4656724ffc20fd69b0ee5bb...,802598005,1,2020-06-10,2,2,2,"[888024001, 802598005]",0.152508,0.084729,104,28.000000,1.000000,802598005,Wednesday,Spring,0.014058
1362277,ffe782e2c7bef7695ef26a06452d7118941e1b5b9811e8...,788928001,3,2019-12-09,9,35,48,"[627762005, 659053002, 633785015, 683787002, 6...",1.631390,0.084729,22,85.250000,5.333333,819520008,Monday,Winter,0.009875
1362278,ffeb9d72ceb9850a485875f390cd0b4e7790580198686b...,750769002,1,2020-01-20,1,5,5,"[599580049, 685605009, 625178001, 590928022, 7...",0.099915,0.030492,246,0.000000,5.000000,599580049,Monday,Winter,0.015850
1362279,ffedc92bc0daea38ca7283ec2dd3b30f475766dbbe3fc4...,832441001,1,2020-05-14,1,3,3,"[863561002, 832441001, 770352006]",0.164695,0.106763,131,0.000000,3.000000,770352006,Thursday,Spring,0.005863


# Trying K means Clustering

In [242]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import RandomizedSearchCV

In [248]:
num_features = list( features.columns[2:-3].append(features.columns[[-1]]))
num_features.remove('list_of_items_bought')
num_features.remove('t_dat')

preprocess = make_column_transformer((StandardScaler(), num_features))

clusterer = Pipeline(
       [
           (
               "kmeans",
               KMeans(
                   n_clusters=1,
                   init="k-means++",
                   n_init=50,
                   max_iter=500,
                   random_state=42,
               ),
           ),
       ]
    )

In [268]:
param_dist = {
          'kmeans__n_clusters': [2,3,4,5]
}


pipe = sklearn.pipeline.make_pipeline(preprocess,PCA(n_components=3, random_state=42),KMeans(n_clusters=1,init="k-means++",n_init=50,max_iter=500,random_state=42))




In [270]:
# Doing randomized search on the models

search = RandomizedSearchCV(pipe, param_distributions = param_dist)
search.fit(features)

C:\Users\ayush\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


KeyboardInterrupt: 

Exception ignored in: 'sklearn.cluster._k_means_common._relocate_empty_clusters_dense'
Traceback (most recent call last):
  File "<__array_function__ internals>", line 2, in where
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
search.best_params_

In [ ]:
#checking if taking first 3 components makes sense
print(pipe['pca'].explained_variance_ratio_)

In [258]:
# SSE
pipe['kmeans'].inertia_

9699384.244318962

In [234]:
# Printing SSE for 3 dimensions
pipe['pipeline']['kmeans'].inertia_

1836050.1822210508

In [213]:
# Predicted Labels
pipe['pipeline']['kmeans'].labels_

array([1, 9, 5, ..., 5, 8, 3])

array([0.41408227, 0.13178738, 0.10139931, 0.08974505, 0.07388963,
       0.0666591 , 0.06055119, 0.04117373, 0.01530884, 0.00458151])

In [274]:
#visualizing the kmeans plot

import matplotlib.pyplot as plt
import seaborn as sns

pcadf = pd.DataFrame(
    pipe['pca'].transform(pipe['columntransformer'].transform(features)),
    columns=["component_1", "component_2"],
)


plt.style.use("fivethirtyeight")
plt.figure(figsize=(8, 8))

scat = sns.scatterplot(
    "component_1",
    "component_2",
    s=50,
    data=pcadf,
    palette="Set2",
)

scat.set_title('H&M predictions')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)

plt.show()

NotFittedError: This PCA instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

# Preprocessing Articles

In [4]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
    
def preprocess_2(articles_data):
    n = 3 # top n TF-IDF words

    tfidf = TfidfVectorizer(token_pattern=r"\w+") # no words are left out
    X = tfidf.fit_transform(articles_data['detail_desc'].values.astype('U'))
    ind = (-X.todense()).argpartition(n)[:, :n]
    top_words = pd.Series(list(np.array(tfidf.get_feature_names())[ind]))

    subset_cols = ['article_id','product_type_no','graphical_appearance_no','colour_group_code','perceived_colour_value_id','perceived_colour_master_id','department_no','index_group_no','section_no', 'garment_group_no']
    articles_data = articles_data.loc[:,subset_cols]
    articles_data['prod_desc'] = top_words
    return articles_data

In [15]:
articles_data = preprocess_2(articles_data)

C:\Users\ayush\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [18]:
a_t = pd.merge(articles_data,transaction_data, on = 'article_id')
final = pd.merge(customers_data,a_t, on ='customer_id' )

In [41]:
sub_sample = final.sample(frac=0.001, replace=False, random_state=1)

# Label Encoding

In [42]:
sub_sample['club_member_status'] = sub_sample.club_member_status.astype("category").cat.codes
sub_sample['fashion_news_frequency'] = sub_sample.fashion_news_frequency.astype("category").cat.codes
sub_sample['postal_code'] = sub_sample.postal_code.astype("category").cat.codes

# SubSamping & Train Test Split 

In [43]:
X = sub_sample.drop(['customer_id','article_id','t_dat','prod_desc'],axis = 1)
y = sub_sample['article_id']




from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=False)

from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train, random_state=0)
X_test, y_test = shuffle(X_test, y_test, random_state=0)



In [44]:
# X_train[['club_member_status']] = enc.fit_transform(X_train[['club_member_status']].values.reshape(-1,1))
# X_train[['fashion_news_frequency']] = enc.fit_transform(X_train[['fashion_news_frequency']].values.reshape(-1,1))

# Random Forest

In [46]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.compose import make_column_transformer
import sklearn
from sklearn.metrics import accuracy_score

clf2 = RandomForestClassifier(n_estimators=50, min_samples_split=5,random_state=0,max_samples=200,n_jobs=-1,bootstrap = True)


#l=5000000
#clf2.fit(X_train[:l], y_train[:l])

# label = ['club_member_status','fashion_news_frequency']

#preprocess = make_column_transformer((OrdinalEncoder(), label),
#                                      remainder = "passthrough") 


pipe = sklearn.pipeline.make_pipeline(clf2)
    
pipe.fit(X_train.fillna(0), y_train)





Pipeline(steps=[('randomforestclassifier',
                 RandomForestClassifier(max_samples=200, min_samples_split=5,
                                        n_estimators=50, n_jobs=-1,
                                        random_state=0))])

In [47]:
import gc

# freeing up memory
del X_train
gc.collect()

104

# Predicting

In [48]:
# X_test[['club_member_status']] = enc.transform(X_test[['club_member_status']].values.reshape(-1,1))
# X_test[['fashion_news_frequency']] = enc.transform(X_test[['fashion_news_frequency']].values.reshape(-1,1))
y_pred = pipe.predict(X_test.fillna(0))
print(accuracy_score(y_test, y_pred))

0.011819654942331522


In [51]:
pipe.predict_proba(X_test.fillna(0))

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])